In [4]:
import sys
import os
from dotenv import load_dotenv

# 1. 환경 변수 로드 (.env 파일 읽기)
load_dotenv()

# (참고) 만약 노트북 파일을 'notebooks' 같은 하위 폴더에 만들었다면 
# 아래 주석을 풀어서 프로젝트 루트 경로를 추가해줘야 합니다.
# sys.path.append(os.path.abspath(".."))

# 2. 우리가 만든 함수 가져오기
# 수정을 했을 때 자동으로 다시 불러오게(Reload) 하는 매직 커맨드입니다.
%load_ext autoreload
%autoreload 2

try:
    from src.tools import get_detailed_route
    print("✅ 함수 불러오기 성공!")
except ImportError as e:
    print(f"❌ 오류 발생: {e}")

❌ 오류 발생: cannot import name 'get_detailed_route' from 'src.tools' (/Users/seoungmun/Documents/work/3-2/project/travle_agent4/src/tools.py)


In [34]:
# [TEST 1] 구글 맵 경로 탐색 테스트
from src.tools import get_detailed_route

# 1. 가까운 거리 (버스/지하철)
print("--- 🚌 단거리 이동 테스트 ---")
short_route = get_detailed_route("서울역", "N서울타워", mode="transit")
if short_route:
    print(f"소요시간: {short_route['duration']}")
    print(f"이동경로: {short_route['steps']}")
else:
    print("❌ 단거리 경로 탐색 실패")

# 2. 먼 거리 (복합 환승)
print("\n--- 🚇 장거리 이동 테스트 ---")
long_route = get_detailed_route("홍대입구역", "롯데월드", mode="transit")
if long_route:
    print(f"소요시간: {long_route['duration']}")
    print(f"이동경로: {long_route['steps']}")
else:
    print("❌ 장거리 경로 탐색 실패")

--- 🚌 단거리 이동 테스트 ---
소요시간: 21분
이동경로: ['[BUS] 402', '🚶 도보 8분']

--- 🚇 장거리 이동 테스트 ---
소요시간: 49분
이동경로: ['[SUBWAY] 2호선', '[SUBWAY] 2호선', '🚶 도보 5분']


In [5]:
# [TEST 2] RAG 검색(Retriever) 테스트
from src.tools import search_attractions_and_reviews

# 1. 구체적인 검색어 테스트
query = "부산 해운대 근처 오션뷰 카페 추천해줘"
print(f"🔎 검색 질문: {query}")

# 도구 실행 (내부적으로 FAISS 검색 -> LLM 요약 과정을 거침)
# 주의: 처음 실행 시 DB 로딩 때문에 시간이 조금 걸릴 수 있습니다.
result = search_attractions_and_reviews.invoke(query)

print("\n[검색 결과]")
print(result)

🔎 검색 질문: 부산 해운대 근처 오션뷰 카페 추천해줘

--- [DEBUG] search_attractions_and_reviews 호출됨 ---
DEBUG: RAG 원본 사용자 쿼리: 부산 해운대 근처 오션뷰 카페 추천해줘
DEBUG: RAG-Fusion 생성 쿼리 (최대 5개): ['부산 해운대 근처에서 바다 전망이 멋진 카페를 추천해 주시겠어요? 사진 찍기 좋은 곳이면 더 좋겠어요.', '부산 해운대 지역에서 가장 인기 있는 오션뷰 카페는 어디인가요? 방문객 리뷰가 좋은 곳 위주로 알려주세요.', '해운대 근처에서 조용하고 여유롭게 바다를 감상할 수 있는 오션뷰 카페를 추천해 주세요. 주차 가능한 곳이면 좋겠습니다.', '해운대 해변 근처 말고, 조금 떨어져 있어도 괜찮으니 전망이 정말 좋은 오션뷰 카페를 알려주세요. 현지인들이 자주 가는 숨겨진 명소도 환영합니다.', '부산 해운대 근처에서 오션뷰도 좋지만, 커피나 디저트 메뉴가 특별히 맛있는 카페를 추천해 주실 수 있나요? 브런치 메뉴가 있는 곳도 좋습니다.']
DEBUG: FAISS 원본 검색 결과 (Raw Docs): [[Document(metadata={'지역': '부산광역시 서구', '카테고리': '카페', '장소명': 'EL16.52', '별점': 3.4, '상세 주소': '부산 서구 암남공원로 177', '리뷰': '어디에 앉든 오션뷰라 사진도 잘나오고 베이커리도 음료도 맛있어요~'}, page_content='지역: 부산광역시 서구 | 장소명: EL16.52 | 카테고리: 카페 | 리뷰: 어디에 앉든 오션뷰라 사진도 잘나오고 베이커리도 음료도 맛있어요~'), Document(metadata={'지역': '부산광역시 서구', '카테고리': '카페', '장소명': '이디야커피 부산송도해상케이블카점', '별점': 3.8, '상세 주소': '부산 서구 송도해변로 171 3층', '리뷰': 'good view! also good coffee! can take a beautiful photo fro

In [1]:
# Jupyter Notebook (디버깅용)

import sys
import os

# 경로 설정
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever

print("🚀 검색 함수 디버깅 시작\n")

# DB 로드
try:
    db = load_faiss_index()
    print("✅ DB 로드 성공")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    exit()

def test_search(query, location):
    print(f"\n🔎 검색 테스트: '{query}' (지역: {location})")
    
    retriever = RegionPreFilteringRetriever(
        vectorstore=db, 
        k=3, 
        fixed_location=location
    )
    
    docs = retriever.invoke(query)
    
    if not docs:
        print("   -> 결과 없음")
        return

    for i, doc in enumerate(docs):
        # 메타데이터에서 지역 정보 추출해 출력
        meta = doc.metadata
        region_info = meta.get('지역') or meta.get('주소') or meta.get('addr') or "정보없음"
        print(f"   {i+1}. [{doc.page_content[:20]}...] (지역정보: {region_info})")

# --- 테스트 케이스 ---

# 1. 부산 송정 맛집 (강원도가 나오면 실패)
test_search("송정 맛집", "부산")

# 2. 서울 카페 (서울특별시 매칭 확인)
test_search("조용한 카페", "서울특별시")

# 3. 엉뚱한 검색 (제주도에서 경복궁 찾기 -> 결과 없어야 정상)
test_search("경복궁", "제주도")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 22:31:33.293 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:33.315 
  command:

    streamlit run

🚀 검색 함수 디버깅 시작



2025-12-04 22:31:33.825 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:33.826 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:33.827 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:40.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:40.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 22:31:40.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ DB 로드 성공

🔎 검색 테스트: '송정 맛집' (지역: 부산)
   1. [지역: 부산광역시 서구 | 장소명: ...] (지역정보: 부산광역시 서구)

🔎 검색 테스트: '조용한 카페' (지역: 서울특별시)
   1. [지역: 서울특별시 성북구 | 장소명:...] (지역정보: 서울특별시 성북구)
   2. [지역: 서울특별시 노원구 | 장소명:...] (지역정보: 서울특별시 노원구)
   3. [지역: 서울특별시 금천구 | 장소명:...] (지역정보: 서울특별시 금천구)

🔎 검색 테스트: '경복궁' (지역: 제주도)
   -> 결과 없음


In [2]:
import random
import pandas as pd
from tqdm import tqdm
import sys
import os
import time

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

try:
    from src.config import load_faiss_index
    from src.search import RegionPreFilteringRetriever
    from src.region_cut_fuzz import normalize_region_name
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")

# --- 1. 테스트 데이터 생성기 (기존 코드 활용) ---
def generate_test_dataset(num_samples=50):
    """50개의 랜덤 지역별 질문 생성 (비교를 위해 샘플 수 조정 가능)"""
    regions = [
        "서울", "부산", "제주", "강릉", "여수", "경주", "전주", "대구", "대전", 
        "춘천", "속초", "포항", "거제", "광주", "울산", "수원", "가평", "양평"
    ]
    templates = [
        "{region} 맛집 추천해줘", "{region} 예쁜 카페", "{region} 가볼만한 곳", 
        "{region} 아이랑 갈만한 곳", "{region} 핫플레이스"
    ]
    
    dataset = []
    for _ in range(num_samples):
        region = random.choice(regions)
        query = random.choice(templates).format(region=region)
        target_region = normalize_region_name(region) # 예: 부산 -> 부산광역시
        
        dataset.append({
            "query": query,
            "target_region_full": target_region, # 정규화된 이름 (필터링용)
            "target_region_short": region # 짧은 이름 (검증용)
        })
    return dataset

# --- 2. 정답 판별 헬퍼 함수 ---
def is_correct_region(doc, target_full, target_short):
    """문서의 메타데이터가 목표 지역과 일치하는지 판별"""
    doc_region = str(doc.metadata.get('지역') or doc.metadata.get('addr') or "")
    
    # 1. 광주광역시 vs 경기도 광주 구분이 제일 중요
    if target_short == "광주" and target_full == "광주광역시":
        if "경기도" in doc_region: return False
        
    # 2. 고성 (강원 vs 경남) 구분
    if target_short == "고성":
        if "강원" in target_full and "경남" in doc_region: return False
        if "경남" in target_full and "강원" in doc_region: return False

    # 3. 일반적인 포함 여부 확인
    # (문서 지역명에 '부산' 혹은 '부산광역시'가 포함되어 있는지)
    clean_target = target_full.replace("특별시", "").replace("광역시", "").replace("특별자치시", "").replace("특별자치도", "").replace("도", "")
    
    if clean_target in doc_region or target_short in doc_region:
        return True
    return False

# --- 3. 비교 실행 함수 ---
def run_comparison():
    print("🔄 FAISS 인덱스 로딩 중...")
    db = load_faiss_index()
    
    test_data = generate_test_dataset(50) # 50개 테스트
    results = []
    
    print(f"🚀 [Baseline vs Current] 성능 비교 시작 (총 {len(test_data)}건)...")
    print("-" * 60)

    total_baseline_accuracy = 0
    total_current_accuracy = 0

    for item in tqdm(test_data):
        query = item['query']
        target_full = item['target_region_full']
        target_short = item['target_region_short']
        
        # ---------------------------------------------------------
        # A. [Baseline] 기존 랭체인 기본 검색 (유사도 기반)
        # ---------------------------------------------------------
        # 단순히 쿼리와 유사한 문서를 가져옴. 지역 필터링 없음.
        baseline_docs = db.similarity_search(query, k=5)
        
        baseline_score = 0
        baseline_regions = []
        for doc in baseline_docs:
            r_name = doc.metadata.get('지역', 'Unknown')[:10] # 너무 길면 자름
            baseline_regions.append(r_name)
            if is_correct_region(doc, target_full, target_short):
                baseline_score += 1
        
        baseline_acc = (baseline_score / 5) * 100 # 5개 중 몇 개나 맞았는지 (%)

        # ---------------------------------------------------------
        # B. [Current] 현재 구현한 선필터링 검색 (Fetch & Filter)
        # ---------------------------------------------------------
        retriever = RegionPreFilteringRetriever(
            vectorstore=db,
            k=5,
            fixed_location=target_full # 목적지 강제 지정
        )
        try:
            current_docs = retriever.invoke(query)
        except:
            current_docs = []

        current_score = 0
        current_regions = []
        for doc in current_docs:
            r_name = doc.metadata.get('지역', 'Unknown')[:10]
            current_regions.append(r_name)
            if is_correct_region(doc, target_full, target_short):
                current_score += 1
        
        # 결과가 0개면 정확도 0 처리 (혹은 필터링 성공으로 볼 수도 있으나 여기선 검색 실패로 간주)
        if len(current_docs) > 0:
            current_acc = (current_score / len(current_docs)) * 100
        else:
            current_acc = 0 # 너무 빡빡해서 결과가 없으면 0점

        # 통계 누적
        total_baseline_accuracy += baseline_acc
        total_current_accuracy += current_acc

        # ---------------------------------------------------------
        # 결과 기록 (차이가 나는 경우만 로그에 강조)
        # ---------------------------------------------------------
        note = ""
        if current_acc > baseline_acc:
            note = "✅ 개선됨"
        elif current_acc < baseline_acc:
            note = "🔻 성능하락"
        
        results.append({
            "Query": query,
            "Target": target_short,
            "Baseline_Regions": str(baseline_regions), # 문자열로 변환해 보기 좋게
            "Baseline_Acc": baseline_acc,
            "Current_Regions": str(current_regions),
            "Current_Acc": current_acc,
            "Note": note
        })

    # --- 4. 결과 리포트 출력 ---
    df = pd.DataFrame(results)
    
    avg_base = total_baseline_accuracy / len(test_data)
    avg_curr = total_current_accuracy / len(test_data)

    print("\n" + "="*60)
    print(f"📊 최종 비교 리포트")
    print(f"🔹 기존 방식(Baseline) 평균 지역 정확도: {avg_base:.2f}%")
    print(f"🔸 현재 방식(Current)  평균 지역 정확도: {avg_curr:.2f}%")
    print(f"📈 성능 향상 폭: +{avg_curr - avg_base:.2f}%p")
    print("="*60)

    # 특히 개선된 케이스 상위 5개 출력
    improved = df[df['Note'] == "✅ 개선됨"].head(10)
    if not improved.empty:
        print("\n[✨ 주요 개선 사례: 엉뚱한 지역이 제거된 경우]")
        # 보기 좋게 컬럼 선택
        print(improved[['Query', 'Baseline_Regions', 'Current_Regions']].to_string(index=False))
    
    return df

# 실행
if __name__ == "__main__":
    df_result = run_comparison()

🔄 FAISS 인덱스 로딩 중...
🚀 [Baseline vs Current] 성능 비교 시작 (총 50건)...
------------------------------------------------------------


100%|██████████| 50/50 [00:16<00:00,  3.08it/s]


📊 최종 비교 리포트
🔹 기존 방식(Baseline) 평균 지역 정확도: 92.00%
🔸 현재 방식(Current)  평균 지역 정확도: 100.00%
📈 성능 향상 폭: +8.00%p

[✨ 주요 개선 사례: 엉뚱한 지역이 제거된 경우]
       Query                                                       Baseline_Regions                                                        Current_Regions
포항 아이랑 갈만한 곳         ['경상북도 포항시', '경상북도 포항시', '경상북도 포항시', '대구광역시 수성구', '대구광역시 수성구']           ['경상북도 포항시', '경상북도 포항시', '경상북도 포항시', '경상북도 포항시', '경상북도 포항시']
    전주 핫플레이스  ['전북특별자치도 전주', '전북특별자치도 전주', '강원특별자치도 양구', '전북특별자치도 전주', '서울특별시 동작구'] ['전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주', '전북특별자치도 전주']
    가평 핫플레이스              ['대구광역시 달성군', '경기도 가평군', '경기도 화성시', '경기도 가평군', '경기도 가평군']                ['경기도 가평군', '경기도 가평군', '경기도 가평군', '경기도 가평군', '경기도 가평군']
  서울 맛집 추천해줘            ['충청북도 단양군', '서울 강북구', '충청북도 단양군', '충청북도 단양군', '서울특별시 동작구']            ['서울 강북구', '서울특별시 동작구', '서울특별시 강남구', '서울특별시 성동구', '서울 마포구']
  울산 맛집 추천해줘          ['울산광역시 울주군', '울산광역시 남구', '울산광역시 중구', '경상북도 청송군', '울산광역시 남구']       

In [6]:
import pandas as pd
import os

# ⚠️ 파일 경로를 본인의 실제 파일명으로 수정하세요!
# 예: "new_reviews.csv" 또는 "data/place_data.csv"
csv_file_path = "/Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv" 

if os.path.exists(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)
        
        # '지역' 컬럼 확인
        if '지역' in df.columns:
            unique_regions = sorted(df['지역'].dropna().unique())
            print(f"✅ 발견된 지역 목록 (총 {len(unique_regions)}개):")
            print("-" * 30)
            
            # 너무 많으면 앞부분만 출력
            for region in unique_regions:
                print(f"- {region}")

                
            print("-" * 30)
            print("💡 확인 포인트: '부산광역시 해운대구' 처럼 '시/도 + 구/군'이 모두 포함되어 있나요?")
        else:
            print(f"❌ '{csv_file_path}' 파일에 '지역' 컬럼이 없습니다. 컬럼 목록: {df.columns}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {csv_file_path}")

✅ 발견된 지역 목록 (총 254개):
------------------------------
- 강원특별자치도 강릉시
- 강원특별자치도 고성군
- 강원특별자치도 동해시
- 강원특별자치도 삼척시
- 강원특별자치도 속초시
- 강원특별자치도 양구군
- 강원특별자치도 양양군
- 강원특별자치도 영월군
- 강원특별자치도 원주시
- 강원특별자치도 인제군
- 강원특별자치도 정선군
- 강원특별자치도 철원군
- 강원특별자치도 춘천시
- 강원특별자치도 태백시
- 강원특별자치도 평창군
- 강원특별자치도 홍천군
- 강원특별자치도 화천군
- 강원특별자치도 횡성군
- 경기도 가평군
- 경기도 고양시
- 경기도 과천시
- 경기도 광명시
- 경기도 광주시
- 경기도 구리시
- 경기도 군포시
- 경기도 김포시
- 경기도 남양주시
- 경기도 동두천시
- 경기도 부천시
- 경기도 성남시
- 경기도 수원시
- 경기도 시흥시
- 경기도 안산시
- 경기도 안성시
- 경기도 안양시
- 경기도 양주시
- 경기도 양평군
- 경기도 여주시
- 경기도 연천군
- 경기도 오산시
- 경기도 용인시
- 경기도 의왕시
- 경기도 의정부시
- 경기도 이천시
- 경기도 파주시
- 경기도 평택시
- 경기도 포천시
- 경기도 하남시
- 경기도 화성시
- 경상남도 거제시
- 경상남도 거창군
- 경상남도 고성군
- 경상남도 김해시
- 경상남도 남해군
- 경상남도 밀양시
- 경상남도 사천시
- 경상남도 산청군
- 경상남도 양산시
- 경상남도 의령군
- 경상남도 진주시
- 경상남도 창녕군
- 경상남도 창원시
- 경상남도 통영시
- 경상남도 하동군
- 경상남도 함안군
- 경상남도 함양군
- 경상남도 합천군
- 경상북도 경산시
- 경상북도 경주시
- 경상북도 고령군
- 경상북도 구미시
- 경상북도 김천시
- 경상북도 문경시
- 경상북도 봉화군
- 경상북도 상주시
- 경상북도 성주군
- 경상북도 안동시
- 경상북도 영덕군
- 경상북도 영양군
- 경상북도 영주시
- 경상북도 영천시
- 경상북도 예천군
- 경상북도 울릉군
- 경상북도 울

In [ ]:
import pandas as pd
import os
import pickle
from src.config import review_faiss

# 1. 확인할 CSV 파일명 (사용자님의 실제 파일명으로 꼭 변경!)
CSV_FILE = "/Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv"

# 2. FAISS 메타데이터 파일 경로 (일반적으로 review_faiss 폴더 안에 있음)
META_PKL_PATH = os.path.join(review_faiss, "metadata_list.pkl")  # 혹은 "index.pkl" 등 저장 방식에 따라 다름

print("🔍 데이터 정밀 진단을 시작합니다...\n")

# --- 진단 1: 원본 CSV 파일 확인 ---
print(f"[진단 1] 원본 CSV 파일 확인: {CSV_FILE}")
if os.path.exists(CSV_FILE):
    try:
        df = pd.read_csv(CSV_FILE)
        print(f"✅ 파일 로드 성공! 총 {len(df)}개 행")
        
        # '지역' 컬럼 확인
        if '지역' in df.columns:
            null_count = df['지역'].isnull().sum()
            print(f"✅ '지역' 컬럼 존재함 (결측치: {null_count}개)")
            
            # 부산 지역 샘플 확인
            busan_sample = df[df['지역'].str.contains("부산", na=False)].head(3)
            if not busan_sample.empty:
                print("\n   [부산 데이터 샘플]")
                for _, row in busan_sample.iterrows():
                    print(f"   - {row['지역']} | {row.get('장소', '이름없음')}")
            else:
                print("⚠️ 경고: '부산'이 포함된 데이터가 CSV에 없습니다.")
        else:
            print(f"❌ 치명적 오류: CSV에 '지역' 컬럼이 없습니다! (현재 컬럼: {list(df.columns)})")
            
    except Exception as e:
        print(f"❌ CSV 파일 읽기 오류: {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {CSV_FILE}")


# --- 진단 2: FAISS 메타데이터 (pickle) 확인 ---
# 이전에 지역 검색이 잘 되었다면, 이 파일이 살아있을 수 있습니다.
print(f"\n[진단 2] FAISS 메타데이터(pickle) 확인: {META_PKL_PATH}")

if os.path.exists(META_PKL_PATH):
    try:
        with open(META_PKL_PATH, "rb") as f:
            meta_list = pickle.load(f)
            
        print(f"✅ 메타데이터 파일 로드 성공! 총 {len(meta_list)}개 항목")
        
        # 첫 번째 데이터 까보기
        if len(meta_list) > 0:
            first_item = meta_list[0]
            print("\n   [첫 번째 메타데이터 구조 확인]")
            print(f"   Type: {type(first_item)}")
            print(f"   Content: {first_item}")
            
            # 우리가 필요한 'region' 키가 있는지 확인
            if isinstance(first_item, dict):
                if 'region' in first_item:
                    print(f"   ✅ 'region' 키가 존재합니다. (값 예시: {first_item['region']})")
                elif '지역' in first_item:
                    print(f"   ⚠️ 'region' 대신 '지역' 키가 있습니다. 리트리버 코드를 수정해야 할 수도 있습니다.")
                else:
                    print(f"   ❌ 'region' 관련 키가 없습니다! (키 목록: {list(first_item.keys())})")
            elif hasattr(first_item, 'metadata'): # Document 객체인 경우
                print(f"   ✅ Document 객체입니다. Metadata: {first_item.metadata}")
            else:
                print("   ⚠️ 알 수 없는 데이터 타입입니다.")
                
    except Exception as e:
        print(f"❌ 메타데이터 파일 읽기 오류: {e}")
else:
    print(f"⚠️ 메타데이터 파일을 찾을 수 없습니다.")
    print("   (참고: FAISS DB 폴더 안에 'index.faiss'만 있고 메타데이터 파일이 분리되어 저장되지 않았을 수도 있습니다.)")

print("\n------------------------------")
print("💡 진단 가이드:")
print("1. [진단 1]에서 '지역' 컬럼이 있고 샘플이 잘 나온다면 -> 원본 데이터는 정상입니다.")
print("2. [진단 2]에서 파일이 없거나 'region' 키가 없다면 -> DB에 메타데이터가 누락된 상태입니다.")
print("   이 경우, 아쉽지만 검색 시 필터링을 할 수 없으므로 DB 재구축(rebuild)이 불가피할 수 있습니다.")

🔍 데이터 정밀 진단을 시작합니다...

[진단 1] 원본 CSV 파일 확인: /Users/seoungmun/Documents/work/3-2/project/travle_agent4/data_csv/metadata_preview.csv
✅ 파일 로드 성공! 총 168591개 행
✅ '지역' 컬럼 존재함 (결측치: 0개)

   [부산 데이터 샘플]
   - 부산광역시 중구 | 이름없음
   - 부산광역시 중구 | 이름없음
   - 부산광역시 중구 | 이름없음

[진단 2] FAISS 메타데이터(pickle) 확인: /Users/seoungmun/Documents/work/3-2/project/travle_agent4/review_faiss/metadata_list.pkl
✅ 메타데이터 파일 로드 성공! 총 168591개 항목

   [첫 번째 메타데이터 구조 확인]
   Type: <class 'dict'>
   Content: {'지역': '부산광역시 중구', '카테고리': '맛집', '장소명': '이정태본참치', '별점': 4.4, '상세 주소': '부산 중구 중앙대로81번길 9 1층', '리뷰': '여기는 점심 굶고 저녁에 와야합니다.\n배찢어져요\n스키다시에 회에 정신못차려요', 'text_for_embedding': '지역: 부산광역시 중구 | 장소명: 이정태본참치 | 카테고리: 맛집 | 리뷰: 여기는 점심 굶고 저녁에 와야합니다.\n배찢어져요\n스키다시에 회에 정신못차려요'}
   ⚠️ 'region' 대신 '지역' 키가 있습니다. 리트리버 코드를 수정해야 할 수도 있습니다.

------------------------------
💡 진단 가이드:
1. [진단 1]에서 '지역' 컬럼이 있고 샘플이 잘 나온다면 -> 원본 데이터는 정상입니다.
2. [진단 2]에서 파일이 없거나 'region' 키가 없다면 -> DB에 메타데이터가 누락된 상태입니다.
   이 경우, 아쉽지만 검색 시 필터링을 할 수 없으므로 DB 재구축(rebuild)이 불가피할 수

In [11]:
# [Cell] 수정된 디버깅 코드 (한글 키 지원)
from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever 

def debug_retriever_v2(target_location, query):
    print(f"\n🧪 [테스트] 목적지: '{target_location}' | 검색어: '{query}'")
    
    db = load_faiss_index()
    
    # 수정된 리트리버 사용
    retriever = RegionPreFilteringRetriever(
        vectorstore=db, 
        k=5, 
        fixed_location=target_location
    )
    
    docs = retriever.invoke(query)
    
    print(f"📄 검색된 문서 수: {len(docs)}개")
    print("-" * 60)
    
    error_count = 0
    for i, doc in enumerate(docs):
        # [핵심 수정] 한글 키 '지역', '장소명'으로 가져오기
        region = doc.metadata.get('지역') or doc.metadata.get('region') or '지역정보 없음'
        place_name = doc.metadata.get('장소명') or doc.metadata.get('place_name') or '이름 없음'
        
        # 검증 로직
        check_keyword = target_location.replace("특별시", "").replace("광역시", "").replace("특별자치시", "").replace("특별자치도", "")
        
        is_valid = check_keyword in region
        status_icon = "✅" if is_valid else "❌ [오류: 타지역]"
        
        if not is_valid: error_count += 1
            
        print(f"{status_icon} {i+1}. [{region}] {place_name}")
        
    print("-" * 60)
    if error_count == 0 and len(docs) > 0:
        print("🎉 테스트 성공: 모든 결과가 정상입니다!")
    elif len(docs) == 0:
        print("⚠️ 검색 결과가 0개입니다. (키워드 매칭 실패 또는 데이터 없음)")
    else:
        print(f"⚠️ 테스트 실패: {error_count}개의 타지역 데이터가 발견되었습니다.")

# --- 실행 ---
debug_retriever_v2("부산광역시", "해운대 오션뷰 카페")
debug_retriever_v2("서울특별시", "강남 맛집")


🧪 [테스트] 목적지: '부산광역시' | 검색어: '해운대 오션뷰 카페'
📄 검색된 문서 수: 3개
------------------------------------------------------------
✅ 1. [부산광역시 서구] 오그남
✅ 2. [부산광역시 서구] 오그남
✅ 3. [부산광역시 해운대구] 스타벅스 하버타운점
------------------------------------------------------------
🎉 테스트 성공: 모든 결과가 정상입니다!

🧪 [테스트] 목적지: '서울특별시' | 검색어: '강남 맛집'
📄 검색된 문서 수: 5개
------------------------------------------------------------
✅ 1. [서울특별시 강남구] 능라도 강남점
✅ 2. [서울특별시 강남구] 창고43 강남점
✅ 3. [서울특별시 강남구] 큰물참치 강남점
✅ 4. [서울특별시 강남구] 큰물참치 강남점
✅ 5. [서울특별시 강남구] 농민백암순대 강남직영점
------------------------------------------------------------
🎉 테스트 성공: 모든 결과가 정상입니다!


In [13]:
import time
import random
import pandas as pd
from tqdm import tqdm

# 프로젝트 모듈 가져오기
from src.config import load_faiss_index
from src.region_cut_fuzz import REGION_STRUCTURE, normalize_region_name
from src.search import RegionPreFilteringRetriever

# DB 로드 (시간이 좀 걸릴 수 있습니다)
print("📂 FAISS 인덱스 로딩 중...")
db = load_faiss_index()
print("✅ DB 로드 완료!")

# 질문 템플릿 (다양한 변주)
TEMPLATES = [
    "{region} 맛집 추천해줘",
    "{region} 가볼 만한 곳 있어?",
    "{region} 예쁜 카페 알려줘",
    "{region} 데이트 코스 추천",
    "{region} 1박 2일 여행지",
    "{region} 아이랑 갈만한 식당",
    "{region} 분위기 좋은 술집",
    "{region} 유명한 빵집",
    "{region} 현지인 맛집 어디야",
    "{region} 주차하기 좋은 카페"
]

def generate_evaluation_dataset(num_samples=1000):
    """
    전국 행정구역과 템플릿을 조합하여 테스트 셋을 생성합니다.
    """
    print(f"⚙️ {num_samples}개의 테스트 쿼리 생성 중...")
    dataset = []
    
    # 1. 테스트할 지역 리스트 준비
    all_targets = []
    for city, districts in REGION_STRUCTURE.items():
        # 광역시/도 단위 (예: 부산광역시)
        all_targets.append((city, city)) 
        # 구/군 단위 (예: 부산광역시 해운대구)
        for dist in districts:
            all_targets.append((city, f"{city} {dist}")) 

    # 2. 랜덤 샘플링
    for _ in range(num_samples):
        # 지역 선택
        target_city, target_detail = random.choice(all_targets)
        
        # 템플릿 선택
        template = random.choice(TEMPLATES)
        
        # 쿼리 생성 (노이즈 추가: 30% 확률로 상위 지역명 생략 -> 기존 모델 헷갈리게 유도)
        # 예: "부산광역시 해운대구" -> "해운대구 맛집"
        if random.random() < 0.3 and target_city != target_detail:
            query_region = target_detail.split()[-1] # "해운대구"
        else:
            query_region = target_detail # "부산광역시 해운대구"
            
        query = template.format(region=query_region)
        
        dataset.append({
            "query": query,
            "target_city": target_city,       # 정답 판별용 (상위)
            "target_detail": target_detail,   # 정답 판별용 (하위)
            "fixed_location": target_city     # 제안 모델 입력용 (세션 정보 가정)
        })
        
    return dataset

# 데이터셋 생성 테스트 (5개만 미리보기)
sample_data = generate_evaluation_dataset(5)
print("\n[생성된 데이터 예시]")
for d in sample_data:
    print(f"- Q: {d['query']} | Target: {d['target_detail']}")

📂 FAISS 인덱스 로딩 중...
✅ DB 로드 완료!
⚙️ 5개의 테스트 쿼리 생성 중...

[생성된 데이터 예시]
- Q: 경기도 평택시 주차하기 좋은 카페 | Target: 경기도 평택시
- Q: 제천시 예쁜 카페 알려줘 | Target: 충청북도 제천시
- Q: 대전광역시 동구 주차하기 좋은 카페 | Target: 대전광역시 동구
- Q: 달서구 데이트 코스 추천 | Target: 대구광역시 달서구
- Q: 인제군 분위기 좋은 술집 | Target: 강원특별자치도 인제군


In [14]:
# [Model A] Baseline: 기존 랭체인 기본 검색 (필터링 X)
class BaselineRetriever:
    def __init__(self, vectorstore, k=5):
        # 랭체인의 기본 as_retriever 사용 (단순 유사도 검색)
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": k})
        
    def invoke(self, query):
        return self.retriever.invoke(query)

# [Model B] Proposed: 제안하는 하이브리드 필터링 검색
# (이미 src.search.RegionPreFilteringRetriever 클래스가 정의되어 있으므로 그대로 사용)

In [ ]:
import time
import pandas as pd
from tqdm import tqdm
import sys
import os

# 경로 설정
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

from src.config import load_faiss_index
from src.search import RegionPreFilteringRetriever
from src.region_cut_fuzz import normalize_region_name

# --- 1. 헷갈리는 지역 데이터셋 생성 (집중 공격) ---
def generate_confusing_dataset():
    """
    이름이 같거나 비슷해서 검색기가 헷갈리기 쉬운 지역들만 모아 테스트셋 생성
    """
    # 1. 중구 (전국 6개 광역시)
    jung_gu_list = [
        "서울특별시 중구", "부산광역시 중구", "대구광역시 중구", 
        "인천광역시 중구", "대전광역시 중구", "울산광역시 중구"
    ]
    
    # 2. 강서구 (서울 vs 부산)
    gangseo_gu_list = ["서울특별시 강서구", "부산광역시 강서구"]
    
    # 3. 서구, 동구, 남구, 북구 (광역시에 공통)
    common_gu_list = [
        "부산광역시 서구", "대구광역시 서구", "인천광역시 서구", "광주광역시 서구", "대전광역시 서구",
        "부산광역시 동구", "대구광역시 동구", "인천광역시 동구", "광주광역시 동구", "대전광역시 동구"
    ]
    
    # 4. 고성 (강원 vs 경남)
    goseong_list = ["강원특별자치도 고성군", "경상남도 고성군"]
    
    # 5. 광주 (광주광역시 vs 경기도 광주시)
    gwangju_list = ["광주광역시", "경기도 광주시"]

    # 전체 리스트 합치기
    target_regions = jung_gu_list + gangseo_gu_list + common_gu_list + goseong_list + gwangju_list
    
    dataset = []
    templates = ["{r} 맛집 추천", "{r} 가볼만한 곳", "{r} 유명한 카페", "{r} 여행 코스"]
    
    for full_region_name in target_regions:
        # 쿼리 생성 (예: "부산 중구 맛집", "강원도 고성 여행")
        # 정규화된 이름에서 시/도, 구/군을 분리하여 자연스러운 쿼리 생성
        parts = full_region_name.split()
        if len(parts) >= 2:
            short_name = f"{parts[0][:2]} {parts[1]}" # 예: "부산 중구"
        else:
            short_name = parts[0]

        for temp in templates:
            query = temp.format(r=short_name)
            dataset.append({
                "query": query,          # 예: "부산 중구 맛집 추천"
                "target": full_region_name # 정답: "부산광역시 중구"
            })
            
    return dataset

# --- 2. 평가 루프 (이전과 동일 로직) ---
def run_stress_test():
    print("📂 FAISS 인덱스 로딩 중...")
    try:
        db = load_faiss_index()
    except Exception as e:
        print(f"❌ DB 로드 실패: {e}")
        return

    # 데이터 준비
    test_data = generate_confusing_dataset()
    print(f"⚠️ [Hard Mode] 동음이의 지역 집중 테스트 셋 생성 완료 (총 {len(test_data)}개)")
    print("   (예시: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)")

    results = {
        "Baseline": {"correct": 0, "total": 0},
        "Proposed": {"correct": 0, "total": 0}
    }
    
    failed_cases_baseline = [] # 베이스라인이 틀린 경우 수집

    print("🚀 평가 시작...")
    
    for item in tqdm(test_data):
        query = item['query']
        target_region = item['target']
        
        # --- (A) Baseline (단순 검색) ---
        base_retriever = db.as_retriever(search_kwargs={"k": 3})
        try:
            base_docs = base_retriever.invoke(query)
        except:
            base_docs = []
        
        # Baseline 채점
        # 검색 결과 3개 중 하나라도 '타 지역'이 섞이면 실패로 간주 (Strict Mode)
        base_is_correct = True
        retrieved_wrong = []
        
        for doc in base_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            
            # [검증 로직] 정답 지역의 시/도 이름이 포함되어 있는지 확인
            # 예: 정답 '부산광역시 중구' -> 문서에 '부산'이 있어야 함. '서울'이 있으면 오답.
            target_sido = target_region.split()[0][:2] # '부산'
            
            if target_sido not in doc_region:
                base_is_correct = False
                retrieved_wrong.append(doc_region)
        
        if base_is_correct and base_docs:
            results["Baseline"]["correct"] += 1
        else:
            # 실패 사례 저장 (분석용)
            if len(failed_cases_baseline) < 5: # 5개만 샘플링
                failed_cases_baseline.append({
                    "Query": query,
                    "Target": target_region,
                    "Wrong_Result": retrieved_wrong[:1] # 하나만 표시
                })

        results["Baseline"]["total"] += 1

        # --- (B) Proposed (필터링 검색) ---
        # 정답 지역명(target_region)을 기반으로 필터링 수행
        prop_retriever = RegionPreFilteringRetriever(
            vectorstore=db, 
            k=3, 
            fixed_location=target_region # 이미 정규화된 정답셋을 넣음
        )
        try:
            prop_docs = prop_retriever.invoke(query)
        except:
            prop_docs = []

        # Proposed 채점
        prop_is_correct = True
        for doc in prop_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2]
            
            if target_sido not in doc_region:
                prop_is_correct = False
        
        if prop_is_correct and prop_docs:
            results["Proposed"]["correct"] += 1
        results["Proposed"]["total"] += 1

    # --- 3. 결과 리포트 ---
    print("\n" + "="*60)
    print("📊 [동음이의 지역] 성능 비교 리포트")
    print("="*60)

    final_stats = []
    for model_name, metrics in results.items():
        acc = (metrics['correct'] / metrics['total']) * 100 if metrics['total'] > 0 else 0
        final_stats.append({
            "모델": model_name,
            "정확도 (Accuracy)": f"{acc:.2f}% ({metrics['correct']}/{metrics['total']})"
        })

    df = pd.DataFrame(final_stats)
    print(df)
    
    print("-" * 60)
    print("📉 Baseline(기존) 주요 실패 사례:")
    for case in failed_cases_baseline:
        print(f" - Q: '{case['Query']}' (목표: {case['Target']})")
        print(f"   -> ❌ 오답 검색됨: {case['Wrong_Result']}")
    print("-" * 60)

# 실행
if __name__ == "__main__":
    run_stress_test()

In [18]:
# 평가 실행 (샘플 수 1000개)
run_comparative_evaluation(1000)

⚙️ 1000개의 테스트 쿼리 생성 중...

🚀 [기존 모델 vs 제안 모델] 비교 평가 시작 (총 1000회)


100%|██████████| 1000/1000 [07:57<00:00,  2.10it/s]


📊 평가 결과 리포트


,모델,지역 정확도 (Precision),평균 속도 (Latency)
0,Baseline (기존),84.96%,235.07 ms
1,Proposed (제안),86.27%,241.34 ms


------------------------------------------------------------
💡 분석:
1. Baseline은 전국 데이터 유사도 검색이므로 동명 지역(예: 중구, 고성)에서 오류가 발생합니다.
2. Proposed는 메타데이터 필터링을 통해 해당 지역 내에서만 검색하므로 정확도가 획기적으로 높습니다.


In [43]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys
import os
import platform

# --- 1. 환경 설정 ---
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# 한글 폰트 설정
if platform.system() == 'Darwin': # Mac
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows
    plt.rc('font', family='Malgun Gothic')
else: # Linux
    plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

try:
    from src.config import load_faiss_index
    from src.search import RegionPreFilteringRetriever
    from src.region_cut_fuzz import normalize_region_name
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")

# --- 2. 헷갈리는 지역 데이터셋 생성 ---
def generate_confusing_dataset():
    # 동음이의 지역 리스트
    target_regions = [
        "서울특별시 중구", "부산광역시 중구", "대구광역시 중구", 
        "인천광역시 중구", "대전광역시 중구", "울산광역시 중구",
        "서울특별시 강서구", "부산광역시 강서구",
        "강원특별자치도 고성군", "경상남도 고성군",
        "광주광역시", "경기도 광주시"
    ]
    
    dataset = []
    templates = ["{r} 맛집 추천", "{r} 가볼만한 곳", "{r} 유명한 카페", "{r} 여행 코스"]
    
    for full_region_name in target_regions:
        parts = full_region_name.split()
        if len(parts) >= 2:
            short_name = f"{parts[0][:2]} {parts[1]}" # 예: "부산 중구"
        else:
            short_name = parts[0]

        for temp in templates:
            dataset.append({
                "query": temp.format(r=short_name), 
                "target": full_region_name
            })
    return dataset

# --- 3. 평가 및 시각화 실행 ---
def run_stress_test_and_visualize():
    print("📂 FAISS 인덱스 로딩 중...")
    try:
        db = load_faiss_index()
    except Exception as e:
        print(f"❌ DB 로드 실패: {e}")
        return

    test_data = generate_confusing_dataset()
    print(f"⚠️ [Hard Mode] 동음이의 지역 집중 테스트 시작 (총 {len(test_data)}회)")
    print("   (Target: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)")

    results = {
        "Baseline": {"correct": 0, "total": 0, "time": []},
        "Proposed": {"correct": 0, "total": 0, "time": []}
    }
    
    for item in tqdm(test_data):
        query = item['query']
        target_region = item['target']
        
        # --- (A) Baseline ---
        start_time = time.time()
        base_retriever = db.as_retriever(search_kwargs={"k": 3})
        try: base_docs = base_retriever.invoke(query)
        except: base_docs = []
        end_time = time.time()
        
        base_is_correct = True
        for doc in base_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2] # '부산'
            if target_sido not in doc_region:
                base_is_correct = False
        
        if base_is_correct and base_docs:
            results["Baseline"]["correct"] += 1
        results["Baseline"]["total"] += 1
        results["Baseline"]["time"].append(end_time - start_time)

        # --- (B) Proposed ---
        start_time = time.time()
        # [핵심] 정답 지역을 미리 알고 필터링한다고 가정 (의도 파악 성공 시)
        prop_retriever = RegionPreFilteringRetriever(vectorstore=db, k=3, fixed_location=target_region)
        try: prop_docs = prop_retriever.invoke(query)
        except: prop_docs = []
        end_time = time.time()

        prop_is_correct = True
        for doc in prop_docs:
            doc_region = str(doc.metadata.get('지역') or doc.metadata.get('region') or "")
            target_sido = target_region.split()[0][:2]
            if target_sido not in doc_region:
                prop_is_correct = False
        
        if prop_is_correct and prop_docs:
            results["Proposed"]["correct"] += 1
        results["Proposed"]["total"] += 1
        results["Proposed"]["time"].append(end_time - start_time)

    # --- 4. 결과 집계 ---
    final_stats = []
    for model_name, metrics in results.items():
        precision = (metrics['correct'] / metrics['total']) * 100 if metrics['total'] > 0 else 0
        avg_latency = (sum(metrics['time']) / len(metrics['time'])) * 1000
        
        # [핵심] 시각화 코드와 키 이름을 통일했습니다.
        final_stats.append({
            "모델": model_name,
            "지역 정확도 (Precision)": precision, 
            "평균 속도 (Latency)": avg_latency
        })

    df_result = pd.DataFrame(final_stats)
    
    # 텍스트 출력용 포맷팅
    df_display = df_result.copy()
    df_display['지역 정확도 (Precision)'] = df_display['지역 정확도 (Precision)'].apply(lambda x: f"{x:.2f}%")
    df_display['평균 속도 (Latency)'] = df_display['평균 속도 (Latency)'].apply(lambda x: f"{x:.2f} ms")
    
    print("\n📊 [동음이의 지역] 평가 결과 리포트")
    print(df_display)

    # --- 5. 시각화 ---
    visualize_stress_results(df_result)

def visualize_stress_results(df_plot):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    sns.set_style("whitegrid")

    # 정확도 그래프
    sns.barplot(x='모델', y='지역 정확도 (Precision)', data=df_plot, ax=axes[0], palette='viridis')
    axes[0].set_title('동음이의 지역 정확도 비교', fontsize=15, fontweight='bold')
    axes[0].set_ylabel('정확도 (%)', fontsize=12)
    axes[0].set_ylim(0, 110)
    for p in axes[0].patches:
        axes[0].annotate(f'{p.get_height():.1f}%', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', fontsize=12, fontweight='bold')

    # 속도 그래프
    sns.barplot(x='모델', y='평균 속도 (Latency)', data=df_plot, ax=axes[1], palette='rocket')
    axes[1].set_title('평균 검색 속도 (Latency)', fontsize=15, fontweight='bold')
    axes[1].set_ylabel('평균 소요 시간 (ms)', fontsize=12)
    for p in axes[1].patches:
        axes[1].annotate(f'{p.get_height():.2f} ms', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()
    
    # CSV 저장
    df_plot.to_csv("stress_test_result.csv", index=False, encoding='utf-8-sig')
    print("💾 결과가 'stress_test_result.csv'로 저장되었습니다.")

# 실행
if __name__ == "__main__":
    run_stress_test_and_visualize()

📂 FAISS 인덱스 로딩 중...
⚠️ [Hard Mode] 동음이의 지역 집중 테스트 시작 (총 48회)
   (Target: 서울 중구 vs 부산 중구, 강원 고성 vs 경남 고성 등)


100%|██████████| 48/48 [00:18<00:00,  2.57it/s]



📊 [동음이의 지역] 평가 결과 리포트
         모델 지역 정확도 (Precision) 평균 속도 (Latency)
0  Baseline             77.08%       211.47 ms
1  Proposed            100.00%       176.97 ms
💾 결과가 'stress_test_result.csv'로 저장되었습니다.


/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_52410/1025229079.py:152: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='모델', y='지역 정확도 (Precision)', data=df_plot, ax=axes[0], palette='viridis')
/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_52410/1025229079.py:160: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='모델', y='평균 속도 (Latency)', data=df_plot, ax=axes[1], palette='rocket')
/var/folders/_f/zr1lq38j0sb4t8xvj43g2khm0000gn/T/ipykernel_52410/1025229079.py:167: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [42]:
import os
import sys
from dotenv import load_dotenv

# 환경 변수 로드 (.env 파일이 루트 폴더에 있다고 가정)
load_dotenv()

# 프로젝트 루트 경로 추가 (src/ 모듈 임포트용)
# 만약 노트북이 src 폴더 밖에 있다면 이 코드가 필요합니다.
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

# 함수 및 모듈 임포트
try:
    from src.tools import search_attractions_and_reviews, get_weather_forecast
    # normalize_region_name은 search_attractions_and_reviews 내부에서 사용됨
    print("✅ 필수 함수 임포트 성공!")
except ImportError as e:
    print(f"❌ 임포트 오류: {e}")
    print("   (src/tools.py 파일이 프로젝트 경로에 저장되었는지 확인해주세요.)")

# RAG 결과가 너무 길어 콘솔 출력을 정리하기 위한 헬퍼 함수
def print_search_result(result):
    if result.startswith("오류"):
        print(f"🚨 오류 발생: {result}")
    else:
        # LLM에게 전달할 최종 프롬프트 (요약된 결과)를 확인합니다.
        print("\n--- LLM 처리 최종 결과 (카테고리/설명 포함) ---")
        print(result)

✅ 필수 함수 임포트 성공!


In [ ]:
import os
import sys
from typing import Dict, Any
from langchain_core.messages import HumanMessage

# 1. 경로 설정 (필수)
# 노트북이 src 폴더 밖에 있다면 이 코드가 src 모듈을 찾게 해줍니다.
# (이전 셀에서 실행했다면 생략 가능)
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Graph Flow 임포트
try:
    from src.graph_flow import build_graph, AgentState
    print("✅ Graph Flow 임포트 성공!")
except ImportError as e:
    print(f"❌ Graph Flow 임포트 오류: {e}")
    print("   (src/graph_flow.py 파일을 확인해주세요.)")
    sys.exit()

# 3. 그래프 인스턴스 생성
app = build_graph()

# 4. 초기 상태 설정 (Streamlit 세션 상태 시뮬레이션)
# '여행 정보 입력' 페이지에서 사용자가 입력하고 저장했다고 가정합니다.

# ⚠️ 여기에 원하는 테스트 값을 입력하세요!
TEST_DESTINATION = "부산"
TEST_DATES = "2025년 12월 5일 ~ 2025년 12월 7일" # 3일치 테스트
TEST_PREFERENCE = "오션뷰 카페와 해산물 맛집 위주"
TEST_TOTAL_DAYS = 3

# Graph Flow의 초기 입력 상태
initial_state_input: AgentState = {
    "messages": [HumanMessage(content="안녕하세요! 입력한 정보로 1일차 계획을 시작해주세요.")],
    "current_weather": "", # 날씨를 비워둬서 WeatherAgent가 호출되도록 유도
    "itinerary": [],
    "destination": TEST_DESTINATION,
    "dates": TEST_DATES,
    "preference": TEST_PREFERENCE,
    "total_days": TEST_TOTAL_DAYS,
    "activity_level": 3,
    "current_planning_day": 1,
    "show_pdf_button": False
}

# 5. 그래프 실행 (WeatherAgent 호출까지 진행)
print("\n=======================================================")
print(f"🚀 Graph Flow 실행 시작 (목적지: {TEST_DESTINATION}, 기간: {TEST_DATES})")
print("=======================================================")

# Graph Flow는 Entry Point에서 WeatherAgent를 선택할 것입니다.
final_state = app.invoke(initial_state_input)

# 6. 디버깅 로그 확인
print("\n[디버깅 로그 검증 포인트]")
print("1. WeatherAgent 호출 시:")
print("   - 'DEBUG_W_1': Agent 전달 목적지가 '부산'으로 정확히 찍혔는가?")
print("   - 'DEBUG_W_2': Agent 전달 기간이 '2025년 12월 5일 ~ 2025년 12월 7일'로 찍혔는가?")
print("   - 'DEBUG_W_3': OWM 데이터에 3일 이상의 날짜 정보가 포함되어 있는가?")
print("2. Graph Flow의 최종 상태:")
print(f"   - 최종 Weather 상태: {final_state.get('current_weather', '업데이트 실패')[:50]}...")

✅ Graph Flow 임포트 성공!

🚀 Graph Flow 실행 시작 (목적지: 부산, 기간: 2025년 12월 5일 ~ 2025년 12월 7일)
--- (Supervisor) 다음 작업 결정 ---
--- (Supervisor) 다음 작업 결정 ---
Supervisor -> SupervisorAgent (ToolMessage 결과 처리 요청)

[디버깅 로그 검증 포인트]
1. WeatherAgent 호출 시:
   - 'DEBUG_W_1': Agent 전달 목적지가 '부산'으로 정확히 찍혔는가?
   - 'DEBUG_W_2': Agent 전달 기간이 '2025년 12월 5일 ~ 2025년 12월 7일'로 찍혔는가?
   - 'DEBUG_W_3': OWM 데이터에 3일 이상의 날짜 정보가 포함되어 있는가?
2. Graph Flow의 최종 상태:
   - 최종 Weather 상태: [부산] (2025년 12월 5일 ~ 2025년 12월 7일) 날씨 정보 (기능 점검 중)...


In [28]:
import os
import sys
from typing import Dict, Any
from langchain_core.messages import HumanMessage

# 경로 설정 (이전과 동일)
# ... (경로 설정 코드는 생략하지만, 이전 셀에서 이미 실행되었다고 가정)

try:
    from src.graph_flow import build_graph, AgentState
    print("✅ Graph Flow 임포트 성공!")
except ImportError as e:
    print(f"❌ Graph Flow 임포트 오류: {e}")
    sys.exit()

# 그래프 인스턴스 생성
app = build_graph()

# 1. 가짜 일정 데이터 (시간 정보가 없는 원본 데이터 시뮬레이션)
TEST_ITINERARY = [
    {'day': 1, 'type': '식당', 'name': '해운대 소문난 암소갈비', 'description': '부산 3대 갈비집으로 유명합니다.'},
    {'day': 1, 'type': '관광지', 'name': '해운대 해변', 'description': '해수욕을 즐기기 좋습니다.'},
    {'day': 1, 'type': '카페', 'name': '웨이브온 카페', 'description': '오션뷰가 끝내주는 곳입니다.'},
]

# 2. Graph Flow의 초기 입력 상태
initial_state_input: AgentState = {
    "messages": [HumanMessage(content="경로 최적화 해줘")], # 👈 [핵심] 최적화 요청
    "current_weather": "맑음",
    "itinerary": TEST_ITINERARY, # 👈 가짜 일정 주입
    "destination": "부산",
    "dates": "2025년 12월 5일",
    "preference": "해변 위주",
    "total_days": 1,
    "activity_level": 3,
    "current_planning_day": 1,
    "show_pdf_button": False
}

print("\n=======================================================")
print(f"🚀 Graph Flow 실행 시작: '경로 최적화' 요청 테스트")
print("=======================================================")

# Graph Flow 실행
final_state = app.invoke(initial_state_input)

# 3. 디버깅 로그 확인
print("\n[디버깅 로그 검증 포인트]")
print("1. '--- (Supervisor) 다음 작업 결정 ---' 로그에서 'plan_itinerary_timeline' 도구가 호출되었는가?")
print("2. 'DEBUG TIME PLANNER' 로그에 LLM이 반환한 'estimated_start_time'이 포함된 JSON이 있는가?")
print("3. 최종 상태의 itinerary에 'estimated_start_time' 키가 추가되었는가? (Task 4 성공 여부)")

# 최종 상태에서 시간 정보 포함 여부 확인
final_itinerary = final_state.get('itinerary', [])
has_time_data = any('estimated_start_time' in item for item in final_itinerary)

print(f"\n[최종 상태 요약]")
print(f"Time Data 포함 여부: {'✅ 성공 (시간 계획 완료)' if has_time_data else '❌ 실패 (시간 계획 누락)'}")
print(f"최종 일정 항목 수: {len(final_itinerary)}")

✅ Graph Flow 임포트 성공!

🚀 Graph Flow 실행 시작: '경로 최적화' 요청 테스트
--- (Supervisor) 다음 작업 결정 ---
Router -> call_tools (도구: ['plan_itinerary_timeline'])

--- [DEBUG TIME PLANNER] 시간 계획 시작 ---
--- (Supervisor) 다음 작업 결정 ---
Supervisor -> SupervisorAgent (ToolMessage 결과 처리 요청)
Router -> END

[디버깅 로그 검증 포인트]
1. '--- (Supervisor) 다음 작업 결정 ---' 로그에서 'plan_itinerary_timeline' 도구가 호출되었는가?
2. 'DEBUG TIME PLANNER' 로그에 LLM이 반환한 'estimated_start_time'이 포함된 JSON이 있는가?
3. 최종 상태의 itinerary에 'estimated_start_time' 키가 추가되었는가? (Task 4 성공 여부)

[최종 상태 요약]
Time Data 포함 여부: ❌ 실패 (시간 계획 누락)
최종 일정 항목 수: 3


In [29]:
import unittest
from unittest.mock import patch, MagicMock
import datetime
import os
import sys

# -------------------------------------------------------------------------
# [설정] 프로젝트 루트 경로 추가
# -------------------------------------------------------------------------
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.tools import get_weather_forecast, optimize_and_get_routes
except ImportError:
    print("⚠️ 'src' 모듈을 찾을 수 없습니다. 현재 작업 경로(os.getcwd())가 프로젝트 루트인지 확인해주세요.")
    print(f"현재 경로: {os.getcwd()}")


# -------------------------------------------------------------------------
# [테스트 클래스]
# -------------------------------------------------------------------------
class TestNewTools(unittest.TestCase):

    @patch('src.tools.requests.get')
    def test_weather_forecast_api_limit(self, mock_get):
        """
        [테스트 1] 날씨 API 기간 초과 시 '한계 보고' 메시지 출력 테스트
        """
        print("\n🧪 [Test 1] 날씨 API 기간 한계 로직 테스트 중...")
        
        future_date = datetime.date.today() + datetime.timedelta(days=100)
        dates_str = f"{future_date.year}년 {future_date.month}월 {future_date.day}일부터 3일간"
        
        with patch.dict(os.environ, {"OWM_API_KEY": "TEST_KEY"}):
            result = get_weather_forecast.invoke({"destination": "서울", "dates": dates_str})
        
        print(f"   👉 결과 요약: {result[:60]}...")
        self.assertIn("[날씨 API 한계 보고]", result)
        self.assertIn("최대 5일 후까지만", result)
        print("✅ [Pass] 날짜 범위 초과 시 안내 메시지 정상 출력 확인!")

    @patch('src.tools.GMAPS_CLIENT')
    def test_optimize_and_get_routes_detail(self, mock_gmaps):
        """
        [테스트 2] 경로 최적화 결과에 '상세 경로(버스/지하철)' 정보 포함 여부 테스트
        """
        print("\n🧪 [Test 2] 상세 경로 최적화 및 정보 포함 여부 테스트 중...")
        
        # 1. Mock 데이터 설정 (Distance Matrix)
        mock_matrix_result = {
            'rows': [
                {'elements': [{'status': 'OK', 'duration': {'value': 0}}, {'status': 'OK', 'duration': {'value': 600}}]}, 
                {'elements': [{'status': 'OK', 'duration': {'value': 600}}, {'status': 'OK', 'duration': {'value': 0}}]} 
            ]
        }
        mock_gmaps.distance_matrix.return_value = mock_matrix_result
        
        # 2. Mock 데이터 설정 (Directions: 상세 이동 경로)
        # ⚠️ [수정됨] vehicle 정보를 line 내부로 이동시켰습니다!
        mock_directions_result = [{
            'legs': [{
                'duration': {'text': '15분'},
                'distance': {'text': '3km'},
                'steps': [
                    {
                        'travel_mode': 'TRANSIT',
                        'transit_details': {
                            'line': {
                                'short_name': '143',
                                # vehicle은 line 안에 있어야 코드가 정상 작동합니다.
                                'vehicle': {'type': 'BUS'} 
                            }
                        }
                    }
                ]
            }]
        }]
        mock_gmaps.directions.return_value = mock_directions_result
        
        # 3. 함수 실행
        places = ["장소A", "장소B"]
        result = optimize_and_get_routes.invoke({"places": places})
        
        # 4. 검증
        print(f"   👉 결과 요약 (일부분):\n{result[:150]}...")
        
        self.assertIn("최적 순서", result)
        self.assertIn("상세 이동 경로", result)
        self.assertIn("[BUS] 143", result)
        print("✅ [Pass] 최적화 결과에 상세 대중교통 정보 포함 확인!")

# -------------------------------------------------------------------------
# [실행]
# -------------------------------------------------------------------------
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..


🧪 [Test 2] 상세 경로 최적화 및 정보 포함 여부 테스트 중...

--- [DEBUG] optimize_and_get_routes (Task 3: 상세 경로 최적화) 호출됨 ---
DEBUG: Input places: ['장소A', '장소B']
DEBUG: Distance Matrix API 호출 시도...
DEBUG: Distance Matrix API 호출 성공.
DEBUG: 최적화된 순서: ['장소A', '장소B']
DEBUG: optimize_and_get_routes (Task 3) 상세 경로 포함 완료.
   👉 결과 요약 (일부분):
--- 🗺️ 최적 경로 제안 (총 2곳) ---
**[최적 순서]**: 장소A → 장소B
**[전체 이동 시간 합]**: 약 10 분 (장소 간 이동 시간의 합)

### 🚌 상세 이동 경로
**장소A** ➡️ **장소B**
   - 예상 소요 시간: 15분 (3km)...
✅ [Pass] 최적화 결과에 상세 대중교통 정보 포함 확인!

🧪 [Test 1] 날씨 API 기간 한계 로직 테스트 중...

--- [DEBUG WEATHER] 날씨 검색 시작 ---
DEBUG_W_1: Agent 전달 목적지: 서울
DEBUG_W_2: Agent 전달 기간: 2026년 3월 13일부터 3일간
   👉 결과 요약: 
[날씨 API 한계 보고]
요청 기간: 2026년 3월 13일부터 3일간 (시작일: 2026-03-13)
...
✅ [Pass] 날짜 범위 초과 시 안내 메시지 정상 출력 확인!



----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [47]:
import pandas as pd
import sys
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.config import review_faiss # DB 경로
    # 방금 수정한 updater에서 함수들을 가져옵니다.
    from src.rag_updater import create_documents_from_df, find_detailed_address
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("src/rag_updater.py 파일이 최신 버전으로 저장되었는지 확인해주세요!")

# --- 1. 가짜 리뷰 데이터 생성 (사용자 입력 시뮬레이션) ---
# 사용자는 '상세 주소'를 입력하지 않았습니다.
mock_data = {
    "지역": ["부산 해운대구"], 
    "장소명": ["한우리가"], 
    "카테고리": ["식당"],
    "리뷰": ["밀면이 정말 시원하고 만두가 맛있어요. 부산역 바로 앞이라 가기 편해요."],
    "평점": [5]
}
df_mock = pd.DataFrame(mock_data)

print("📝 [Step 1] 사용자 입력 데이터 (상세 주소 없음):")
display(df_mock) # 또는 print(df_mock)

# --- 2. 실제 FAISS DB 로드 (참조용) ---
print("\n🔄 [Step 2] 기존 지식 베이스(FAISS) 로드 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)
try:
    db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)
    print("✅ DB 로드 성공!")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    # DB가 없으면 테스트 불가하므로 중단
    raise e

# --- 3. 변환 로직 실행 (핵심 검증) ---
print("\n⚙️ [Step 3] 문서 변환 및 주소 자동 보정 실행...")

# create_documents_from_df 함수가 내부적으로 find_detailed_address를 호출합니다.
docs = create_documents_from_df(df_mock, db)

# --- 4. 결과 확인 ---
if docs:
    doc = docs[0]
    print("\n🎉 [Step 4] 변환 결과 확인")
    print("=" * 60)
    
    # 1. 상세 주소가 채워졌는지 확인
    generated_address = doc.metadata.get("상세 주소")
    print(f"📍 [검증 1] 상세 주소 자동 채움 여부:")
    if generated_address:
        print(f"   ✅ 성공! 찾아낸 주소: '{generated_address}'")
    else:
        print(f"   ⚠️ 실패/미발견. (DB에 '한우리가' 데이터가 없거나 매칭 실패)")

    # 2. 텍스트 포맷이 맞는지 확인
    print(f"\n📄 [검증 2] 임베딩용 텍스트 포맷 (page_content):")
    print(f"   {doc.page_content[:100]}...") # 앞부분만 출력
    
    # 포맷 검사
    expected_format = "지역: 부산 해운대구 | 장소명: 한우리가 |"
    if expected_format in doc.page_content:
        print(f"   ✅ 성공! 기존 DB 포맷과 일치합니다.")
    else:
        print(f"   ❌ 실패. 포맷이 다릅니다.")
        
    print("=" * 60)
    print("💡 이 'doc' 객체가 그대로 벡터 DB에 add_documents() 됩니다.")
else:
    print("❌ 문서 생성 실패.")

📝 [Step 1] 사용자 입력 데이터 (상세 주소 없음):


,지역,장소명,카테고리,리뷰,평점
0,부산 해운대구,한우리가,식당,밀면이 정말 시원하고 만두가 맛있어요. 부산역 바로 앞이라 가기 편해요.,5



🔄 [Step 2] 기존 지식 베이스(FAISS) 로드 중...
✅ DB 로드 성공!

⚙️ [Step 3] 문서 변환 및 주소 자동 보정 실행...

🎉 [Step 4] 변환 결과 확인
📍 [검증 1] 상세 주소 자동 채움 여부:
   ⚠️ 실패/미발견. (DB에 '한우리가' 데이터가 없거나 매칭 실패)

📄 [검증 2] 임베딩용 텍스트 포맷 (page_content):
   지역: 부산 해운대구 | 장소명: 한우리가 | 카테고리: 식당 | 리뷰: 밀면이 정말 시원하고 만두가 맛있어요 부산역 바로 앞이라 가기 편해요...
   ✅ 성공! 기존 DB 포맷과 일치합니다.
💡 이 'doc' 객체가 그대로 벡터 DB에 add_documents() 됩니다.


In [50]:
import pandas as pd
import sys
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# [설정] 프로젝트 루트 경로 추가
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

try:
    from src.config import review_faiss # DB 경로
except ImportError as e:
    print(f"❌ 설정 파일 로드 실패: {e}")

# --- 1. FAISS DB 로드 ---
print("🔄 기존 지식 베이스(FAISS) 로드 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="upskyy/bge-m3-korean",
    model_kwargs={"device": "cpu"}
)

try:
    db = FAISS.load_local(review_faiss, embeddings, allow_dangerous_deserialization=True)
    print(f"✅ DB 로드 성공! (총 문서 수: {db.index.ntotal}개)")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    sys.exit()

# --- 2. 검색 및 매칭 테스트 함수 ---
def test_search_and_match(place_name, region):
    print(f"\n🕵️‍♂️ [검색 테스트] 목표: {place_name} ({region})")
    print("-" * 60)
    
    # 2-1. 유사도 검색 수행
    # k를 넉넉하게(10개) 잡아서 비슷한 건 다 가져와 봅니다.
    results = db.similarity_search_with_score(place_name, k=10)
    
    if not results:
        print("   ❌ 검색 결과 없음 (유사한 장소명조차 DB에 없습니다.)")
        return

    found_exact_match = False
    
    print(f"   🔎 유사도 상위 {len(results)}개 결과 분석:")
    for i, (doc, score) in enumerate(results):
        meta = doc.metadata
        db_name = str(meta.get("장소명", "Unknown"))
        db_region = str(meta.get("지역", "Unknown"))
        db_address = str(meta.get("상세 주소", "없음"))
        
        # 공백 제거 후 비교 (매칭 로직 시뮬레이션)
        norm_input_name = place_name.replace(" ", "")
        norm_db_name = db_name.replace(" ", "")
        norm_input_region = region.replace(" ", "")
        norm_db_region = db_region.replace(" ", "")
        
        # 매칭 여부 판단
        name_match = (norm_input_name in norm_db_name) or (norm_db_name in norm_input_name)
        region_match = (norm_input_region in norm_db_region) or (norm_db_region in norm_input_region)
        
        match_status = "❌ 불일치"
        if name_match:
            if region_match:
                match_status = "✅ 일치 (정답 후보)"
                found_exact_match = True
            else:
                match_status = "⚠️ 이름만 같음 (지역 다름)"
        
        print(f"   [{i+1}] {db_name} | {db_region}")
        print(f"       - 상세주소: {db_address}")
        print(f"       - 유사도 거리: {score:.4f} (낮을수록 좋음)")
        print(f"       - 판정: {match_status}")
        
    print("-" * 60)
    if found_exact_match:
        print("🎉 결론: DB에 데이터가 있고, 로직상 찾아낼 수 있습니다.")
    else:
        print("💧 결론: 검색은 되지만, '이름'과 '지역'이 모두 맞는 데이터가 없습니다.")
        print("   -> 상세 주소를 못 채우는 것이 정상입니다. (사용자가 직접 입력해야 함)")

# --- 3. 테스트 실행 ---

# Case A: 아까 실패했던 케이스 (한우리가 / 부산 해운대구)
# -> DB에 없을 가능성이 큼
test_search_and_match("양산왕돼지국밥", "부산광역시 해운대구")

# Case B: DB에 확실히 있을 법한 유명 맛집 (초량밀면 / 부산 동구)
# -> 로직 검증용 (이게 성공하면 로직은 문제 없는 것)
test_search_and_match("초량밀면", "부산 동구")

🔄 기존 지식 베이스(FAISS) 로드 중...
✅ DB 로드 성공! (총 문서 수: 168591개)

🕵️‍♂️ [검색 테스트] 목표: 양산왕돼지국밥 (부산광역시 해운대구)
------------------------------------------------------------
   🔎 유사도 상위 10개 결과 분석:
   [1] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 521.5074 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [2] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 534.6143 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [3] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 546.5441 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [4] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 558.8123 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [5] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 559.4098 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [6] 양산왕돼지국밥 | 부산광역시 해운대구
       - 상세주소: 부산 해운대구 재반로 70 1층
       - 유사도 거리: 570.8265 (낮을수록 좋음)
       - 판정: ✅ 일치 (정답 후보)
   [7] 남도돼지국밥 | 울산광역시 울주군
       - 상세주소: 울산 울주군 범서읍 장검길 9-9 1층
       - 유사도 거리: 

In [1]:
# graph_debug_notebook.ipynb (Jupyter Notebook 셀 1)

import sys
import os

# 프로젝트 루트 경로 설정 (이 노트북 파일이 프로젝트 루트에 있다고 가정)
# 만약 노트북이 src/ 폴더 외부에 있다면, sys.path에 src 폴더를 추가합니다.
if 'src' not in os.listdir('.') and 'src' not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

from langchain_core.messages import HumanMessage, AIMessage
from src.graph_flow import build_graph, AgentState # build_graph 함수 임포트
from datetime import datetime

# Streamlit 의존성 우회 (config.py 및 1_trip_planner.py에서 사용됨)
# Jupyter 환경에서는 st.cache_resource를 사용할 수 없으므로,
# config.py가 FAISS를 로드할 때 오류가 나지 않도록 임시로 함수를 정의합니다.
# 실제 Streamlit 앱에서는 이 코드가 필요 없습니다.
class MockStCache:
    def clear(self):
        pass
    def __call__(self, func):
        return func

try:
    import streamlit as st
except ImportError:
    # Streamlit이 설치되지 않았거나 Jupyter 환경일 경우 Mock 객체 사용
    class MockStreamlit:
        cache_resource = MockStCache()
        def __init__(self):
            # st.error, st.toast 등이 오류를 일으키지 않도록 모킹
            self.error = lambda x: print(f"[MOCK ST.ERROR]: {x}")
            self.toast = lambda x, icon=None: print(f"[MOCK ST.TOAST]: {x}")
            self.stop = lambda: print("[MOCK ST.STOP] Execution Halted.")
            
    st = MockStreamlit()
    sys.modules['streamlit'] = st
    # 1_trip_planner.py에 있는 st.session_state 초기화를 위해 임시 상태 객체를 만듭니다.
    class MockSessionState(dict):
        def get(self, key, default=None):
            return self.get(key, default)
    st.session_state = MockSessionState()

# LangGraph 앱 로드
app = build_graph()
print("✅ LangGraph 애플리케이션 로드 및 컴파일 완료.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ LangGraph 애플리케이션 로드 및 컴파일 완료.


In [9]:
!uv pip install grandalf


Using Python 3.10.0 environment at: /opt/homebrew/Caskroom/miniforge/base/envs/env10
Audited 1 package in 7ms


In [2]:
# Jupyter Notebook 셀 2
print("\n## 🔍 그래프 상태 (AgentState) 분석")
for key, type_hint in AgentState.__annotations__.items():
    print(f"* **{key}**: {type_hint}")

print("\n## 🧭 그래프 노드 및 엣지 분석")
display(app.get_graph().print_ascii())


## 🔍 그래프 상태 (AgentState) 분석
* **messages**: typing.Annotated[list, <function _add_messages_wrapper.<locals>._add_messages at 0x10d880280>]
* **current_weather**: <class 'str'>
* **itinerary**: typing.List[typing.Dict]
* **destination**: <class 'str'>
* **dates**: <class 'str'>
* **preference**: <class 'str'>
* **total_days**: <class 'int'>
* **activity_level**: <class 'int'>
* **current_planning_day**: <class 'int'>
* **show_pdf_button**: <class 'bool'>
* **next_node**: typing.Literal['InfoCollectorAgent', 'WeatherAgent', 'AttractionAgent', 'RestaurantAgent', 'DayTransitionAgent', 'ConfirmationAgent', 'PDFCreationAgent', 'end_node']

## 🧭 그래프 노드 및 엣지 분석


ValueError: no intersection found (point inside ?!). view: <langchain_core.runnables.graph_ascii.VertexViewer object at 0x115188340> topt: (92.5, 13.5)

In [3]:
# Jupyter Notebook 셀 3: 워크플로 실행 및 상태 변화 추적

from datetime import datetime, timedelta # NameError 방지 (셀 1에 있어야 하지만 안전을 위해 다시 포함)

# --- 1. 초기 상태 설정 ---
# 현재 날짜 기준 (초기 HumanMessage에서 사용)
today = datetime.now().strftime('%Y년 %m월 %d일')
# total_days를 2로 설정했으므로, dates도 2일간으로 맞춰줍니다.
dates_str = f"{today}부터 2일간" 

initial_state = {
    "messages": [
        HumanMessage(content=f"""
        안녕하세요! 방금 입력한 정보를 바탕으로 여행 계획을 시작해주세요.
        목적지: 제주도, 여행 기간: {dates_str}
        나의 여행 스타일 및 요청사항: 해산물을 좋아하며, 가족과 함께 가는 여행입니다.
        이제 위 정보를 바탕으로 1일차 계획 추천을 시작해주세요.
        """)
    ],
    "destination": "제주도",
    "dates": dates_str, 
    "preference": "해산물을 좋아하며, 가족과 함께 가는 여행입니다.",
    "total_days": 2,
    "activity_level": 3,
    "current_planning_day": 1,
    "itinerary": [],
    "current_weather": "", 
    "show_pdf_button": False
}

# --- 2. 그래프 실행 및 상태 변화 추적 ---

print("\n## 🚀 워크플로 실행 및 상태 변화 추적 (제주도 맛집)")
print("--------------------------------------------------")

current_state = initial_state
step = 0

# 🚨 get_state() 및 invoke()에 사용할 config를 정의합니다 (ValueError 방지)
CONFIG = {"configurable": {"thread_id": "debug_planner_session"}} 

for step in range(5): # 최대 5단계 실행
    print(f"\n--- [Step {step + 1}] 그래프 실행 시작 ---")
    
    # 1. 그래프 실행 (invoke 시 config 전달)
    try:
        # invoke 호출 시 config를 전달해야 상태가 체크포인터에 저장됩니다.
        output = app.invoke(current_state, config=CONFIG)
    except Exception as e:
        print(f"!!!!!!!!!! [ERROR] 그래프 실행 중 예외 발생: {e} !!!!!!!!!!")
        break

    # 2. 상태 업데이트
    current_state = output

    # 3. 마지막 메시지 출력 (노드에서 나온 결과)
    last_message = current_state['messages'][-1]
    last_message_type = type(last_message).__name__
    # content가 None일 경우 안전하게 처리합니다 (TypeError 방지)
    last_message_content = getattr(last_message, 'content', 'NONE')
    display_content = str(last_message_content) if last_message_content is not None else "NONE"
    
    print(f"--- [Step {step + 1}] 결과 수신 및 상태 업데이트 ---")
    print(f"라우팅 결정: (콘솔 위의 '--- (Supervisor) 다음 작업 결정 ---' 로그 확인)")
    print(f"최종 메시지 타입: **{last_message_type}**")
    print(f"메시지 내용 (요약): {display_content[:100]}...")
    
    # 4. 특정 상태 변화 확인
    if current_state.get('current_weather'):
        print(f"상태 변화: 날씨 정보 채워짐: {current_state['current_weather'][:30]}...")

    # 5. 그래프 종료 조건 확인 (LangGraph 최신 API)
    try:
        # 🚨 get_state 호출 시 config를 전달합니다 (ValueError 방지)
        current_graph_state = app.get_state(CONFIG)
    except Exception as e:
        print(f"!!!!!!!!!! [ERROR] app.get_state 호출 중 오류: {e} (이전 단계 오류의 영향 가능성) !!!!!!!!!!")
        break
        
    # .next_steps가 비어있는지 확인하여 END 도달 여부 판단
    if not current_graph_state.next:
        print("\n--- [TERMINATED] 그래프 실행 완료 (END 노드 도달) ---")
        break
    
    if step == 4:
         print("\n--- [MAX STEPS] 최대 단계 도달. 수동 종료. ---")


## 🚀 워크플로 실행 및 상태 변화 추적 (제주도 맛집)
--------------------------------------------------

--- [Step 1] 그래프 실행 시작 ---
--- (Supervisor) 다음 작업 결정 - 하이브리드 로직 적용 ---
Router -> WeatherAgent (날씨 정보 부족)
Router -> call_tools (도구: ['get_weather_forecast'])

--- [DEBUG] Tool Execution Complete ---
DEBUG: Tool: get_weather_forecast
DEBUG: Arguments: {'dates': '2025년 12월 04일부터 2일간', 'destination': '제주도'}
DEBUG: Output (ToolMessage Content): [제주도 (2025-12-05) 날씨 예보 (OWM)]
- 00:00 (UTC): -0.2℃, 튼구름
- 03:00 (UTC): 1.0℃, 구름조금
- 06:00 (UTC): 1.4℃, 약간의 구름이 낀 하늘
- 09:00 (UTC): -0.0℃, 맑음
- 12:00 (UTC): -0.3℃, 맑음
- 15:00 (UTC): 0.3℃, 약간의 구름이 낀 하늘
- 18:00 (UTC): -0.1℃, 구름조금
- 21:00 (UTC): -0.8℃, 맑음
...
-----------------------------------------
--- (Supervisor) 다음 작업 결정 - 하이브리드 로직 적용 ---
Router -> SupervisorAgent (ToolMessage 결과 브리핑 요청)


2025-12-04 19:39:28.258 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:28.290 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-04 19:39:28.291 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:28.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:28.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Router -> call_tools (도구: ['search_attractions_and_reviews'])

--- [DEBUG] search_attractions_and_reviews 호출됨 ---
DEBUG: RAG 원본 사용자 쿼리: 제주도 가족 해산물 식당


2025-12-04 19:39:28.802 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:28.803 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:28.803 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:37.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:37.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 19:39:37.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DEBUG: RAG-Fusion 생성 쿼리 (최대 5개): ['제주도 아이와 함께 가기 좋은 해산물 식당 추천', '제주도에서 가족 단위로 방문하기 좋은 가성비 해산물 맛집은 어디인가요?', '제주도 오션뷰가 멋진 가족 해산물 식당 추천해주세요.', '제주도에서 신선한 해산물 요리를 맛볼 수 있는 가족 친화적인 식당 리스트를 알려주세요.', '제주도 가족 여행객을 위한 평점 높은 해산물 전문점 정보를 찾아주세요.']
DEBUG: FAISS 원본 검색 결과 (Raw Docs): [[Document(metadata={'지역': '제주특별자치도 제주시', '카테고리': '맛집', '장소명': '고집돌우럭 함덕점', '별점': 4.8, '상세 주소': '제주특별자치도 제주시 조천읍 신북로 491-9 2층', '리뷰': '아이 데리고 왔는데 정말 맛있고 양도 많고 친절하셨어요 \n주변 소개의사 & 재방문 의사 있습니다 :)'}, page_content='지역: 제주특별자치도 제주시 | 장소명: 고집돌우럭 함덕점 | 카테고리: 맛집 | 리뷰: 아이 데리고 왔는데 정말 맛있고 양도 많고 친절하셨어요 \n주변 소개의사 & 재방문 의사 있습니다 :)'), Document(metadata={'지역': '제주특별자치도 서귀포시', '카테고리': '맛집', '장소명': '국수바다 본점', '별점': 4.0, '상세 주소': '제주특별자치도 서귀포시 일주서로 580 1층', '리뷰': '솔직히 기대 하나도 안 했는데\n맛있었음.\n막 엄청 맛있다. 특별하다 그런 거 아니지만\n이 정도면 꽤 괜찮은 수준.\n제주도에서 애들이 제일 맛있게 먹은 곳 ㅎㅎ'}, page_content='지역: 제주특별자치도 서귀포시 | 장소명: 국수바다 본점 | 카테고리: 맛집 | 리뷰: 솔직히 기대 하나도 안 했는데\n맛있었음.\n막 엄청 맛있다. 특별하다 그런 거 아니지만\n이 정도면 꽤 괜찮은 수준.\n제주도에서 애들이 제일 맛있게 먹은 곳 ㅎㅎ'), Document(metadata={'지역': '제

KeyboardInterrupt: 